# Projet : Optimisation des tournées de livraison avec contraintes

---

## Objectif du projet

L’objectif de ce projet est de modéliser et de résoudre un **problème de tournées de véhicules avec capacité (CVRP)**.  
Nous cherchons à **minimiser la distance totale parcourue** tout en respectant les contraintes de capacité des camions et en assurant le passage unique chez chaque

Notre travail se réalise dans une démarche de **mobilité durable**, en lien avec les problématiques actuelles d’optimisation des livraisons pour réduire les coûts et les émissions de CO₂.

---
## Contexte
<p>
  L’ADEME a récemment lancé un appel à manifestation d’intérêt visant à promouvoir le développement de nouvelles solutions de mobilité durable, destinées aux personnes et aux marchandises, et adaptées à différents types de territoires.
  </p>
  <p>
  Dans ce cadre, les enjeux majeurs consistent à réduire les coûts tout en limitant les émissions liées aux transports. 
  Notre équipe <strong>CesiCDP</strong> a répondu à cet appel. Plus précisément, nous cherchons à concevoir un algorithme d’optimisation capable de déterminer, sur un réseau routier donné, une tournée reliant un ensemble de clients à partir d’un seul dépôt, puis revenant à ce point de départ, de manière à minimiser le temps.
  </p>

---

## Introduction
Ce livrable présente la **modélisation mathématique** du problème de tournées de véhicules (VRP). Nous parlerons des variantes réalistes (fenêtres de temps, flotte hétérogène, trafic dépendant du temps), de la démonstration synthétique de la complexité, et des méthodes de résolution pertinentes (exactes, heuristiques, métaheuristiques).

Le but est d’expliquer de manière rigoureuse comment formaliser le problème, quelles formulations peuvent être employées en Recherche Opérationnelle, et quels outils ou solveurs sont les plus appropriés selon la taille des instances, allant de 50 à 2000 clients.

---
## Problématique

> Dans le cadre de la <strong>transition écologique</strong>, comment modéliser et résoudre notre problème de tournées de livraison sur un réseau routier soumis aux contraintes imposées?

> Compte tenu du caractère <strong>combinatoire</strong> du problème, quelles approches issues de la <em>Recherche Opérationnelle</em> peuvent permettre d’obtenir des solutions proches de l’optimal pour des instances de grande taille (plusieurs centaines à milliers de clients) dans des temps de calcul raisonnables ?

---
# Modélisation en Recherche Opérationnelle
## 1. Données et notations
Soit un graphe complet $G=(V,A)$ avec :
- $V=\{0,1,\dots,n\}$ : ensemble des nœuds, où $0$ est le dépôt et $C=V\setminus\{0\}$ l'ensemble des clients.
- $d_{ij}\ge 0$ : distance allant du nœud $i$ au nœud $j$.
  - Les coordonnées sont $(x_i,y_i)$ et type `EUC_2D` :
    $$ c_{ij} = \mathrm{round}\big(\sqrt{(x_i-x_j)^2+(y_i-y_j)^2}\big) $$
- $q_i\ge 0$ : demande du client $i$ (avec $d_0=0$).
- $Q$ : capacité du véhicule.
- $K$ : nombre de véhicules disponibles (optionnel).
-  $[a_i,b_i]$ : fenêtres temporelles 
- $s_i$ : temps de service
- $c_{ij}(t)$ : fonction de trafic

---
## 2. Formulation de base : CVRP (formulation flux / arêtes)
 ### **Variables :** 

$ x_{ij} = \begin{cases}
1 & \text{si le véhicule va directement du client } i \text{ vers } j \\
0 & \text{sinon}
\end{cases} $

$ y_i  = \text {charge totale livrée avant d’arriver au client } i $

$ d_{ij}  = \text {distance (ou coût) entre les clients i  et  j} $

$ q_i = \text {demande du client  i }$

$ Q  = \text {capacité maximale d’un véhicule} $

$ n  = \text {nombre total de clients (hors dépôt)}$

 ### **Fonction économique** 

$
\text{Minimiser } Z = \sum_{i=0}^{n} \sum_{j=0}^{n} d_{ij} \cdot x_{ij}
$

L’objectif est de minimiser la distance totale parcourue par l’ensemble des véhicules.

### **Contraintes**

1. **Chaque client est visité une seule fois :**
$
\sum_{j=0}^{n} x_{ij} = 1 \quad \forall i = 1, \ldots, n
$

2. **Chaque client a exactement un véhicule qui arrive :**
$
\sum_{i=0}^{n} x_{ij} = 1 \quad \forall j = 1, \ldots, n
$

3. **Flux de continuité (chaque véhicule part et revient au dépôt) :**
$
\sum_{j=1}^{n} x_{0j} = k \quad \text{et} \quad \sum_{i=1}^{n} x_{i0} = k
$

4. **Capacité maximale du véhicule :**
$
y_i + q_j \leq Q + (Q - q_i)(1 - x_{ij}) \quad \forall i, j
$

5. **Sous-tournées interdites (MTZ) :**
$
y_i - y_j + Q \cdot x_{ij} \leq Q - q_j \quad \forall i \neq j
$

6. **Nature des variables :**
$
x_{ij} \in \{0,1\}, \quad y_i \geq 0
$

---
## 3. Variantes réalistes
### 3.1 VRPTW — fenêtres temporelles
$$ a_i \le t_i \le b_i $$
$$ t_j \ge t_i + s_i + c_{ij} - M(1 - x_{ij}) $$

### 3.2 Flotte hétérogène
$$ \min \sum_k\sum_{i,j} c_{ij} x_{ij}^k $$

### 3.3 Trafic temps-dépendant $c_{ij}(t)$
- Discrétisation temporelle
- Graphe time-expanded

---
## 4. Complexité
- Cas $K=1$ ⇒ TSP métrique ⇒ NP-complet.
- Donc CVRP est NP-difficile.
→ Nécessité d’heuristiques.

---
## 5. Méthodes de résolution
### 5.1 Exactes
- MILP (Gurobi / CPLEX / SCIP)
- Column Generation + Branch-and-Price

### 5.2 Heuristiques & Métaheuristiques
- Tabu Search
- Simulated Annealing
- Genetic / Memetic
- ALNS (recommandé)
- Hybrides (ALNS + 2-opt, relocate, exchange, cross)

---
## 6. Plan expérimental
- Instances : 50–2000 clients
- 20 runs indépendants
- Métriques : coût, écart, temps, convergence
- Visualisations : boxplots, courbes, scatter
- Analyses : Wilcoxon, t-test
- Reproductibilité : `requirements.txt`, seeds, scripts

---
## 7. Implémentation
- `numpy` pour la matrice de distances
- Éviter copies profondes
- Numba/Cython/C++ pour grandes instances
- Paralléliser runs (multi-cœurs)

---
## 8. Références
- Toth & Vigo (2002). *The Vehicle Routing Problem*. SIAM.
- Lenstra & Kan (1981). *Complexity of vehicle routing problems*. *Networks*.
- Ropke & Pisinger (2006). *An Adaptive Large Neighborhood Search heuristic for the pickup and delivery problem with time windows*. *Transportation Science*.